# Grundlagen der Künstlichen Intelligenz - Wintersemester 2023/24

# Übung 2: Regression und Klassifikation

---

> 'Grundlagen der künstlichen Intelligenz' im Wintersemester 2023/2024
>
> - T.T.-Prof. Pascal Friederich, pascal.friederich@kit.edu
> - Prof. Gerhard Neumann, gerhard.neumann@kit.edu

---

In dieser Übung beschäftigen wir uns genauer mit Regression und Klassifikation, die wir sowohl mithilfe von klassischen Modellen als auch mittels neuronaler Netze in Form mehrlagiger Perzeptrone (*multi-layer perceptrons*, MLPs) betrachten werden. Wir werden dabei sehen, wie wir Methoden des maschinellen Lernens nutzen können, um zu erkennen, welche Informationen in Daten enthalten sind und wie wir diese nutzen können, um Vorhersagen über Phänomene in der realen Welt zu machen.

### Übungsteam
- Philipp Dahlinger, philipp.dahlinger@kit.edu
- Vaisakh Shaj Kumar, v.shaj@kit.edu
- Tobias Schlöder, tobias.schloeder@kit.edu
- Henrik Schopmans, henrik.schopmans@kit.edu

# Auto-grading

Wir nutzen ein auto-grading System, welches eure abgegebenen Jupyter Notebooks automatisch analysiert und über
hidden tests auf Richtigkeit prüft. Über diese Tests werden die Punkte bestimmt, die ihr für das Übungsblatt erhaltet.

Damit das auto-grading reibungslos funktioniert bitte folgende Dinge beachten:

- Notebook muss Dateinamen "ex_02_regression_classification.ipynb" haben
- PDF und Jupyter notebook einzeln im Ilias hochladen (nicht als Zip!)
- Vor dem Abgeben eines Notebooks bitte testen, dass alles von vorne bis hinten ohne Fehler durchläuft.
- Zellen, welche mit "### DO NOT CHANGE ###" markiert sind dürfen weder gelöscht noch bearbeitet werden
- Eure Lösung muss in die richtige Zelle (markiert mit "# YOUR CODE HERE") eingetragen werden.
    - (dabei natürlich den NotImplementedError löschen!)
- Es gibt potentiell scheinbar leere Zellen, die auch mit "### DO NOT CHANGE ###" markiert sind. Auch diese dürfen nicht bearbeitet oder gelöscht werden.
    - Falls dies doch gemacht wird, dann wird das automatische Grading nicht funktionieren und ihr erhaltet keine Punkte.
    - Wir werden hier strikt handeln und keine Ausnahmen machen, falls jemand doch Zellen verändert, die eindeutig als readonly markiert sind!
- Die Jupyter Notebooks haben inline Tests (für euch sichtbar), welche euer Ergebnis auf grobe Richtigkeit überprüfen.
    - Diese sind primär für euch, um Fehler zu erkennen und zu korrigieren.
    - Die inline Tests, die ihr im Notebook sehen könnt, sind allerdings nicht die Tests welche für das Grading verwendet werden!
    - Die inline Tests sind eine notwendige Bedingung, um beim Grading der Aufgabe Punkte zu erhalten!

In [1]:
### DO NOT CHANGE ###

import os
import pickle

import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor

# just one convenience function here ...

def metrics(y_true, y_pred, names):
    functions = dict(
        mse=sklearn.metrics.mean_squared_error,
        r2=sklearn.metrics.r2_score
    )
    return {n: functions[n](y_true, y_pred).round(4) for n in names}

# we set seeds for reproducibility. in practice, a model that works with one
# seed but not with another is a poor and unstable model.
np.random.seed(42)

### DO NOT CHANGE ###

# Aufgabe 2.0 - Einführung und Feature Engineering
In dieser Aufgabe arbeiten wir mit einem auf [Kaggle](https://www.kaggle.com/datasets/ruthgn/wine-quality-data-set-red-white-wine) veröffentlichten Datensatz über Weinqualität.

In [ ]:
### DO NOT CHANGE ###

red_white_wine_pkl = os.path.abspath("red_white_wine.pkl")
if not os.path.exists(red_white_wine_pkl):
    data = pd.read_csv("https://bwsyncandshare.kit.edu/s/AQSTLRDKgjXRxg7/download/wine-quality-white-and-red.csv")
    with open(red_white_wine_pkl, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(red_white_wine_pkl, "rb") as f:
    data = pickle.load(f)

### DO NOT CHANGE ###

Wir arbeiten nur mit dem Teildatensatz zu Rotweinen. Natürlich könnt Ihr zusätzlich testen, wie sich der andere Teildatensatz verhält.

In [ ]:
### DO NOT CHANGE ###

data = data[data.type == 'red']
data = data.drop(['type'], axis = 1)

### DO NOT CHANGE ###

Als erstes versuchen wir, uns einen groben Eindruck davon zu verschaffen, was die Daten darstellen. Dies ist bei einem neuen bzw. unbekannten Datensatz immer empfehlenswert.

In [ ]:
print(data.head(3))

Wir sehen hier eine lange Liste von Features und Werten, wobei noch unklar ist, welche Features wir eine sinnvolle Auswahl darstellen. Unser Label in dieser Aufgabe ist der pH-Wert (Label `pH`), und wir interessieren uns dafür, welche Features dieses Label am besten beschreiben. Da wir noch nicht wissen, wie gut die einzelnen Features zur Erstellung eines Modells verwendet werden können, betrachten wir als erstes Korrelationen in den Daten. Wir möchten dann diejenigen Features auswählen, die einen großen Einfluß auf unser Label haben, die also mit dem pH-Wert korrelieren. Beachtet dabei, dass wir die Beträge der Korrelationskoeffizienten verwenden, da wir uns für den absoluten Einfluss der Korrelation interessieren und nicht für das Vorzeichen. Ein gutes Machine-Learning Modell wird dies ohnehin berücksichtigen, indem es die Parameter entsprechend anpasst.

In [ ]:
pyplot.figure(figsize=(15, 12))
sns.heatmap(data.corr().abs(), cmap="Blues",annot = True)
pyplot.show()

Aus dieser Darstellung können wir zwei Arten von Information erkennen:

Wir können leicht erkennen, welche Features mit dem pH-Wert des Rotweins korrelieren (9. Zeile bzw. Spalte) und sehen zum Beispiel dass die Features `fixed acidity` und `citric acid` einen großen Einfluss auf den pH-Wert zu haben scheinen. Dies ist eine logische und in diesem Fall vermutlich auch richtige Schlussfolgerung, was aber [nicht immer](https://de.wikipedia.org/wiki/Cum_hoc_ergo_propter_hoc) der Fall sein muss. Nach diesem ersten Schritt können wir nun versuchen, die Anzahl an Features zu reduzieren, indem wie nur diejenigen auswählen, die uns aussagekräftig erscheinen.

Des weiteren können wir Korrelationen zwischen den Features ausnutzen, um die Anzahl der ausgewählten Features weiter zu verringern. Das Feature `density` korreliert besipielsweise sowohl mit `fixed acidity` als auch mit `alcohol`.  Obwohl diese Features alle den pH-Wert beschreiben, sind sie doch nicht unabhängig voneinander. Wir können also `density` aus dem Satz der gewählten Features entfernen, da es keine wichtige Zusatzinformation enthält.

Das beschriebene Vorgehen heißt **Feature Engineering**, und muss immer auf die eine oder andere Art durchgeführt werden. Wir sollten jetzt ein besseres Gefühl dafür haben, was in unserem Datensatz steckt und können im nächsten Teil dazu übergehen, die Daten zu nutzen, um etwas Nützliches zu lernen.

# Aufgabe 2.1 - Regression
In diesem Teil geht es erst mit klassischer linearer Regression los und dann mit beliebig komplexen neuronalen Netzen weiter, wobei sich Komplexität hier auf die Modellgröße und nicht auf die Architektur bezieht.

Im ersten Schritt bereiten wir unsere Daten weiter auf und wählen diejenigen Features aus, von dem wir denken, dass die das gewählte Label gut beschreiben und plotten noch einmal die Korrelationen zwischen diesen ausgewählten Features.

In [ ]:
### DO NOT CHANGE ###

x_fields = ["citric acid", "fixed acidity", "volatile acidity", "chlorides", "alcohol"]
y_fields = ["pH"]

n_train = int(len(data) * 0.8)

x_data = data[x_fields]
y_data = data[y_fields]

x_train, x_test = x_data[:n_train].values, x_data[n_train:].values
y_train, y_test = y_data[:n_train].values, y_data[n_train:].values

### DO NOT CHANGE ###

In [ ]:
pyplot.figure(figsize=(15, 12))
sns.heatmap(pd.concat([x_data, y_data], axis=1).corr().abs(), cmap="Blues", annot = True)
pyplot.show()

Wir fangen mit einer einfachen linearen Regression an und verwenden dafür [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) von scikit-learn. Fittet im nächsten Schritt zunächst das Regressionsmodell mit den Trainingsdaten und analysiert dann die Modellparameter. Alle wichtigen Informationen dazu findet Ihr in der Dokumentation.

In [ ]:
lr = LinearRegression()

# fit the model
# YOUR CODE HERE
raise NotImplementedError()

Wie viele Parameter hat das lineare Regressionsmodell? Und welches Feature (Name der Stringvariable) hat den größten Einfluß? Verwendet hierfür den genauen Name der Spalte im Datensatz!

In [ ]:
lr_number_of_parameters: int = None
lr_most_influential_feature: str = None
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
### DO NOT CHANGE ###

assert getattr(lr, "coef_") is not None, "model is not trained"

### DO NOT CHANGE ###

Bei einem einfachen Modell wie der linearen Regression sind die Parameter vergleichsweise leicht interpretierbar, so dass wir uns vorstellen können, was die Parameter bedeuten und welcher Zusammenhang zwischen den Features und dem Label besteht. Allerdings sagt dies noch nichts über die Güte des Modell aus, die auf verschiedene Arten bestimmt werden kann:

In [ ]:
metrics_train = metrics(y_train, lr.predict(x_train), names=["mse", "r2"])
metrics_test = metrics(y_test, lr.predict(x_test), names=["mse", "r2"])
print('Trainingsmetriken: %s'%(str(metrics_train)))
print('Testmetriken:      %s'%(str(metrics_test)))

Als Optimierunsgkriterium wird oft die mittlere quadratische Abweichung (*mean squared error*, MSE) verwendet. Da wir die Labels (pH-Werte) nicht normiert haben, ist MSE allerdings hier kein aussagekräftiges Gütemaß für unser Modell. Es ist daher üblich, andere Statistiken zur Beschreibung der Güte eines Modells zu verwenden. Dabei kommt häufig das [Bestimmmtheitsmaß](https://de.wikipedia.org/wiki/Bestimmtheitsma%C3%9F) (auch $R^2$ score genannt) zum Einsatz, welches Werte im Bereich $[-\infty, 1]$ annehmen kann. Ein $R^2$ score von $0$ bedeutet dabei, dass ein Modell nur den Mittelwert vorhersagt, sodass nur ein Wert von $R^2>0$ anzeigt, dass das Modell etwas gelernt hat.

In unserem Fall beträgt der $R^2$ score $\approx0.5$, und es besteht somit noch Verbesserungspotential. Im nächsten Schritt sehen wir uns im Vergleich ein Neuronales Netz an.

Implementiert nun unter Verwendung von `MLPRegressor` aus scikit-learn eine Funktion, die ein neuronales Netzwerk für Regressionen erstellt.

In [ ]:
def mlp_regressor(
        hidden_units: list, hidden_activation: str
):

    # YOUR CODE HERE
    raise NotImplementedError()

    return model

Konfiguriert als nächstes euer MLP und probiert dabei verschiedene Einstellungen (z.B. Anzahl der Schichten, Aktivierungsfunktion, ...) aus. Trainiert anschließend das Modell mit dem Ziel, eine Konfiguration zu finden, mit dem Euer Modell auf dem Testdatensatz einen $R^2$ score von **größer als 0.5** erzielt.

Bitte fügt der Funktion `multi_layer_perceptron` dabei keine Argumente hinzu und entfernt auch keine.

In [ ]:
regr = mlp_regressor(
    # YOUR CODE HERE
    raise NotImplementedError()
)

Im folgenden Feld werden einige übliche Einstellungen gesetzt, um die Konvergenz zu bechleunigen und die Dauer des Trainings zu verkürzen.

Verändert diese Einstellungen deswegen bitte nicht!

In [ ]:
### DO NOT CHANGE ###

regr.set_params(early_stopping = True, solver='adam', learning_rate='adaptive', max_iter = 2500, verbose = True)

### DO NOT CHANGE ###

In [ ]:
### DO NOT CHANGE ###

print("training model")

regr.fit(x_train, y_train)

### DO NOT CHANGE ###

In [ ]:
metrics_train = metrics(y_train, regr.predict(x_train), names=["mse", "r2"])
metrics_test = metrics(y_test, regr.predict(x_test), names=["mse", "r2"])
print('Trainingsmetriken: %s'%(str(metrics_train)))
print('Testmetriken:      %s'%(str(metrics_test)))

In [ ]:
### DO NOT CHANGE ###

assert metrics_test["r2"] > 0.50, "R^2 should be above 0.50"

### DO NOT CHANGE ###

# Task 2.2 Classification

Im diesen Teil werden wir schließlich die Qualität des (Rot)weines mithilfe der anderen schon oben verwendeten Features vorhersagen. Dazu definieren wir im ersten Schritt drei Qualitätsstufen (niedrig, mittel und hoch) als Labels.

In [ ]:
### DO NOT CHANGE ###

data["quality label"] = data['quality'].apply(lambda x: 0 if x<=5 else 1 if x<=6 else 2) # low, medium, high
y_data = data['quality label'].values

y_train, y_test = y_data[:n_train], y_data[n_train:]

### DO NOT CHANGE ###

Implementiert nun in Analogie zum vorherigen Teil eine Funktion zum Erstellen eines neuronalen Netzes für Klassifikationen!

In [ ]:
def mlp_classifier(
        hidden_units: list, hidden_activation: str
):

    # YOUR CODE HERE
    raise NotImplementedError()

    return model

In [ ]:
clas = mlp_classifier(
    # YOUR CODE HERE
    raise NotImplementedError()
)

In [ ]:
### DO NOT CHANGE ###

clas.set_params(early_stopping = True, solver='adam', learning_rate='adaptive', max_iter = 2500, verbose = True)

### DO NOT CHANGE ###

In [ ]:
### DO NOT CHANGE ###

clas.fit(x_train, y_train)

### DO NOT CHANGE ###

Nachdem Ihr nun ein Modell zur Klassifikation trainiert habt, möchten wir bestimmen, wie gut es ist. Es gibt allerdings keine allgemeingültige Metrik für die Genauigkeit (siehe z.B. [hier](https://scikit-learn.org/stable/modules/model_evaluation.html)).

Hier definieren wir die Genauigkeit als die Anzahl der richtigen Vorhersagen geteilt durch die Gesamtzahl an Vorhersagen. Implementiert dafür im nächsten Feld eine Funktion, die bevorzugt Vektoren benutzt (anstelle von einer Schleife).

In [ ]:
def accuracy(y_true, y_pred):
    # YOUR CODE HERE
    raise NotImplementedError()

    return frac

In [ ]:
acc_train = accuracy(clas.predict(x_train), y_train)
acc_test = accuracy(clas.predict(x_test), y_test)
print("Trainingsgenauigkeit:", acc_train)
print("Testgenauigkeit:     ", acc_test)

# TIPP: Prinizipiell kann man hier auch clas.score(x_train, y_train) verwenden und muss accuracy nicht selbst implementieren.

In [ ]:
### DO NOT CHANGE ###

assert (accuracy(np.array([1.0,2.0,1.0]), np.array([2.0,1.0,1.0])) - 0.33) < 0.01, "wrong accuracy calculation!"


### DO NOT CHANGE ###

In [ ]:
### DO NOT CHANGE ###

assert accuracy(y_test, clas.predict(x_test)) > 0.5, "model accuracy should be above 50%!"


### DO NOT CHANGE ###

In der Praxis wird die Genauigkeit nur selten als einzige Metrik verwendet wird. Betrachten wir dazu als Beispiel einen sehr unausgeglichenen Datensatz mit zwei Klassen, in dem Klasse A mit 99% stark überrepräsentiert ist. Hier kann eine Genauigkeit von 99% nur dadurch erreicht werden, das ein Modell immer Klasse A vorhersagt, sodass die Genauigkeit in diesem Fall offensichtlich nicht zur Beschreibung der Güte des Modells geeignet ist.

Dies ist das Ende der Programmieraufgabe. **Vergesst nicht, auch die Fragen auf dem Übungsblatt zu beantworten** ;)